In [0]:
paths = [
    "/Volumes/workspace/new_data/data/address (1) 1.csv",
    "/Volumes/workspace/new_data/data/drivers (1) 1.csv",
    "/Volumes/workspace/new_data/data/orders (1) 1.csv",
    "/Volumes/workspace/new_data/data/restaurants (2) 1.csv"
]

for path in paths:
    df = spark.read.format("csv").option("header", True).option("delimiter", ",").option("inferSchema", True).load(path)
    display(df.limit(5))

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schemas = {
    "/Volumes/workspace/new_data/data/address (1) 1.csv": StructType([
        StructField("address_id", IntegerType(), True),
        StructField("street", StringType(), True),
        StructField("city", StringType(), True),
        StructField("state", StringType(), True),
        StructField("zip", StringType(), True)
    ]),
    "/Volumes/workspace/new_data/data/drivers (1) 1.csv": StructType([
        StructField("driver_id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("license_number", StringType(), True),
        StructField("vehicle_type", StringType(), True)
    ]),
    "/Volumes/workspace/new_data/data/orders (1) 1.csv": StructType([
        StructField("order_id", IntegerType(), True),
        StructField("customer_id", IntegerType(), True),
        StructField("restaurant_id", IntegerType(), True),
        StructField("driver_id", IntegerType(), True),
        StructField("order_total", DoubleType(), True),
        StructField("order_date", StringType(), True)
    ]),
    "/Volumes/workspace/new_data/data/restaurants (2) 1.csv": StructType([
        StructField("restaurant_id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("cuisine", StringType(), True),
        StructField("address_id", IntegerType(), True)
    ])
}

for path in paths:
    schema = schemas.get(path)
    df = spark.read.format("csv").option("header", True).option("delimiter", ",").schema(schema).load(path)
    print(f"Schema for {path}:")
    df.printSchema()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.new_data.newbronze;

In [0]:
for path in paths:
    schema = schemas.get(path)
    df = spark.read.format("csv").option("header", True).schema(schema).load(path)
    table_name = path.split("/")[-1].replace(" ", "_").replace("(", "").replace(")", "").replace(".csv", "")
    df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("workspace.new_data.newbronze")

In [0]:
%sql
select * from workspace.new_data.newbronze